In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [3]:
# with open("the-verdict.txt" , 'r') as file:
#     data = file.read()
data = input("Enter your prompt here: ")

In [4]:
print(len(data))

534


In [5]:
split_text = re.split(r'([,.<>/!@#$%^&*():;]|--|\s)',data)

In [ ]:
#This a for loop for separating the text. And below is list comprehension

# result = []
# for item in text:
#     if item.strip():
#         result.append(item)
    
# result

In [6]:
result = [item.strip() for item in split_text if item.strip()]
print(result)

['Poor', 'Jack', '!', 'It', 'had', 'always', 'been', 'his', 'fate', 'to', 'have', 'women', 'say', 'such', 'things', 'of', 'him', ':', 'the', 'fact', 'should', 'be', 'set', 'down', 'in', 'extenuation', '.', 'What', 'struck', 'me', 'now', 'was', 'that', ',', 'for', 'the', 'first', 'time', ',', 'he', 'resented', 'the', 'tone', '.', 'I', 'had', 'seen', 'him', ',', 'so', 'often', ',', 'basking', 'under', 'similar', 'tributes', '--', 'was', 'it', 'the', 'conjugal', 'note', 'that', 'robbed', 'them', 'of', 'their', 'savour?', 'No', '--', 'for', ',', 'oddly', 'enough', ',', 'it', 'became', 'apparent', 'that', 'he', 'was', 'fond', 'of', 'Mrs', '.', 'Gisburn', '--', 'fond', 'enough', 'not', 'to', 'see', 'her', 'absurdity', '.', 'It', 'was', 'his', 'own', 'absurdity', 'he', 'seemed', 'to', 'be', 'wincing', 'under', '--', 'his', 'own', 'attitude', 'as', 'an', 'object', 'for', 'garlands', 'and', 'incense', '.']


In [7]:
len(result)

118

In [8]:
words = sorted(set(result))
print(len(words))

78


In [9]:
token_id = {token:i for i,token in enumerate(words)}
print(token_id)

{'!': 0, ',': 1, '--': 2, '.': 3, ':': 4, 'Gisburn': 5, 'I': 6, 'It': 7, 'Jack': 8, 'Mrs': 9, 'No': 10, 'Poor': 11, 'What': 12, 'absurdity': 13, 'always': 14, 'an': 15, 'and': 16, 'apparent': 17, 'as': 18, 'attitude': 19, 'basking': 20, 'be': 21, 'became': 22, 'been': 23, 'conjugal': 24, 'down': 25, 'enough': 26, 'extenuation': 27, 'fact': 28, 'fate': 29, 'first': 30, 'fond': 31, 'for': 32, 'garlands': 33, 'had': 34, 'have': 35, 'he': 36, 'her': 37, 'him': 38, 'his': 39, 'in': 40, 'incense': 41, 'it': 42, 'me': 43, 'not': 44, 'note': 45, 'now': 46, 'object': 47, 'oddly': 48, 'of': 49, 'often': 50, 'own': 51, 'resented': 52, 'robbed': 53, 'savour?': 54, 'say': 55, 'see': 56, 'seemed': 57, 'seen': 58, 'set': 59, 'should': 60, 'similar': 61, 'so': 62, 'struck': 63, 'such': 64, 'that': 65, 'the': 66, 'their': 67, 'them': 68, 'things': 69, 'time': 70, 'to': 71, 'tone': 72, 'tributes': 73, 'under': 74, 'was': 75, 'wincing': 76, 'women': 77}
